In [1]:
import pandas as pd
import numpy as np

## Data loading

In [2]:
bank_data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',names=['A'+str(i) for i in range(1,17)])
bank_data.describe(include=['object', 'float', 'int'])

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
count,690,690,690.000000,690,690,690,690,690.000000,690,690,690.00000,690,690,690,690.000000,690
unique,3,350,NaN,4,4,15,10,NaN,2,2,NaN,2,3,171,NaN,2
top,b,?,NaN,u,g,c,v,NaN,t,f,NaN,f,g,00000,NaN,-
freq,468,12,NaN,519,519,137,399,NaN,361,395,NaN,374,625,132,NaN,383
mean,NaN,NaN,4.758725,NaN,NaN,NaN,NaN,2.223406,NaN,NaN,2.40000,NaN,NaN,NaN,1017.385507,NaN
std,NaN,NaN,4.978163,NaN,NaN,NaN,NaN,3.346513,NaN,NaN,4.86294,NaN,NaN,NaN,5210.102598,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.165000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.00000,NaN,NaN,NaN,5.000000,NaN
75%,NaN,NaN,7.207500,NaN,NaN,NaN,NaN,2.625000,NaN,NaN,3.00000,NaN,NaN,NaN,395.500000,NaN


In [3]:
diabetes_data = pd.read_csv('https://raw.githubusercontent.com/praisan/hello-world/master/diabetes.csv')
diabetes_data.describe(include=['object', 'float', 'int'])

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## Missing value

In [4]:
from sklearn.base import TransformerMixin
class ModeMedianImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with median of column.

        """
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].mode() if X[c].dtype == np.dtype('O') else X[c].median() for c in X],index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [15]:
class BankDataCastType(TransformerMixin):
  def transform(self, X, y=None):
        X=X.replace('?',np.nan)
        X.A2=X.A2.astype(float)
        X.A3=X.A3.astype(float)
        X.A8=X.A8.astype(float)
        X.A11=X.A11.astype(float)
        X.A14=X.A14.astype(float)
        X.A15=X.A15.astype(float)
        return X

In [6]:
diabetes_data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']]=diabetes_data[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0,np.nan)

In [8]:
diabetes_data.describe(include=['object', 'float', 'int'])

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## Standardization

In [9]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
normalize_zscore=StandardScaler(with_mean=True, with_std=True)
normalize_minmax=MinMaxScaler(feature_range=(0, 1))

## Train-Test

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Train test split
x = diabetes_data.drop(['Outcome'], axis=1)
y=np.ravel(diabetes_data['Outcome'])
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=50)

In [16]:
# Train test split
bank_data['A16']=bank_data['A16'].map({'-':1,'+':0})
x = bank_data.drop(['A16'], axis=1)
y=np.ravel(bank_data['A16'])
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=50)

## Decision Tree Classifier

In [ ]:
# ตัวอย่าง classifier ที่มีข้นตอนของการทำ standardization แล้วใช้ logistic regression 
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
missing_model=ModeMedianImputer()
classifier_dt=DecisionTreeClassifier(criterion='gini')

models = Pipeline([
  ('missing', missing_model),
  ('zscore', normalize_zscore),
  ('dt_model', classifier_dt)
])

# ฝึกโมเดลใน Pipeline ด้วยข้อมูลชุดฝึกสอน
_=models.fit(x_train, y_train)

#### Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# นำ Pipeline มาทำนายข้อมูล

y_predict_new_data = models.predict(x_test)

# วัดประสิทธิภาพ

print(classification_report(y_test, y_predict_new_data))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_new_data, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Negative', 'Positive'], columns=['Predicted Negative', 'Predicted Positive'])
print('Confusion matrix')
confusion

              precision    recall  f1-score   support

           0       0.77      0.82      0.79       101
           1       0.61      0.53      0.57        53

    accuracy                           0.72       154
   macro avg       0.69      0.68      0.68       154
weighted avg       0.71      0.72      0.72       154

Confusion matrix


,Predicted Negative,Predicted Positive
Negative,83,18
Positive,25,28


In [ ]:
def show_performance(models,x_test):
  # นำ Pipeline มาทำนายข้อมูล
  y_predict_new_data = models.predict(x_test)
  # วัดประสิทธิภาพ
  print(classification_report(y_test, y_predict_new_data))
  cm_scaled = np.array(confusion_matrix(y_test, y_predict_new_data, labels=[0,1]))
  confusion = pd.DataFrame(cm_scaled, index=['Negative', 'Positive'], columns=['Predicted Negative', 'Predicted Positive'])
  print('Confusion matrix')
  print(confusion)

#### Other model

In [ ]:
classifier_dt_entropy=DecisionTreeClassifier()

In [ ]:
models = Pipeline([
  ('missing', missing_model),
  ('zscore', normalize_zscore),
  ('dt_model', classifier_dt_entropy)
])

# ฝึกโมเดลใน Pipeline ด้วยข้อมูลชุดฝึกสอน
_=models.fit(x_train, y_train)

In [ ]:
show_performance(models,x_test)

              precision    recall  f1-score   support

           0       0.76      0.86      0.81       101
           1       0.64      0.47      0.54        53

    accuracy                           0.73       154
   macro avg       0.70      0.67      0.67       154
weighted avg       0.72      0.73      0.72       154

Confusion matrix
          Predicted Negative  Predicted Positive
Negative                  87                  14
Positive                  28                  25


#### Parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
  {'criterion': ['entropy','gini']}
 ]

In [ ]:
 prediction_model = GridSearchCV(
        DecisionTreeClassifier(class_weight={0:1,1:2}), 
        param_grid, 
        scoring='f1_micro',
        cv=5)

In [ ]:
models = Pipeline([
  ('missing', missing_model),
  ('zscore', normalize_zscore),
  ('dt_model', prediction_model)
])
_=models.fit(x_train, y_train)
show_performance(models,x_test)

              precision    recall  f1-score   support

           0       0.77      0.74      0.75       101
           1       0.54      0.57      0.55        53

    accuracy                           0.68       154
   macro avg       0.65      0.65      0.65       154
weighted avg       0.69      0.68      0.68       154

Confusion matrix
          Predicted Negative  Predicted Positive
Negative                  75                  26
Positive                  23                  30


In [ ]:
print('Best DT model')
print('n_neighbors: '+str(models.steps[2][1].best_estimator_.criterion))

Best DT model
n_neighbors: entropy


---
# ใช้ Model อื่นที่สามารถนำมาใช้ในการแยกประเภทได้


* [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html#classification)

 * ตัวอย่างการใช้
```
from sklearn.svm import SVC
model = SVC(kernel='rbf')
```
 * ตัวอย่างการใช้กับ Pipeline
```
from sklearn.svm import SVC
models = Pipeline([
  ('zscore', StandardScaler()),
  ('c_model', SVC(kernel='rbf'))
```
* [Neural Network](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#classification)
 * ตัวอย่างการใช้
```
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
```
 * ตัวอย่างการใช้กับ Pipeline
```
from sklearn.neural_network import MLPClassifier
models = Pipeline([
  ('zscore', StandardScaler()),
  ('c_model', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1))
```



